In [1]:
from pathlib import Path

import torch
from models.common import DetectMultiBackend
from utils.general import check_img_size, Profile, increment_path
from utils.dataloaders import IMG_FORMATS, LoadImages

In [2]:
device=torch.device('cuda' if torch.cuda.is_available() else 'cpu')
weights = '/work/src/yolov9/weights/yolov9-c-motsynth-48e-best.pt'
dnn = False
data = '/work/src/yolov9/data/yolo-anatomy.yaml'
half = True
imgsz = 640

In [3]:
model = DetectMultiBackend(weights, device=device, dnn=dnn, data=data, fp16=half)

Fusing layers... 
yolov9-c summary: 604 layers, 50698278 parameters, 0 gradients, 236.6 GFLOPs


In [4]:
stride, names, pt = model.stride, model.names, model.pt
imgsz = check_img_size(imgsz, s=stride)

In [5]:
imgsz

640

In [6]:
bs = 1
vid_stride = 1
source = '/work/dataset/VR-DRONE-v1.0.0.test/20221207/1Class/30m/30c/Crop_0030_030m_30c_3_4_00344.jpg'
dataset = LoadImages(source, img_size=imgsz, stride=stride, auto=pt, vid_stride=vid_stride)

In [7]:
model.warmup(imgsz=(bs, 3, imgsz, imgsz))

In [8]:
project = '/work/src/yolov9/runs'
name = 'vis'
visualize = True
exist_ok = True
save_dir = increment_path(Path(project) / name, exist_ok=exist_ok)

augment = False
dt = Profile(), Profile(), Profile()
for path, im, im0s, vid_cap, s in dataset:
    with dt[0]:
        im = torch.from_numpy(im).to(model.device)
        im = im.half() if model.fp16 else im.float()
        im /= 255
        if len(im.shape) == 3:
            im = im[None]
    visualize = increment_path(save_dir/Path(path).stem, mkdir=True) if visualize else False
    pred = model(im, augment=augment, visualize=visualize)
    pred = pred[0][1]

Saving /work/src/yolov9/runs/vis/Crop_0030_030m_30c_3_4_0034411/stage0_Silence_features.png... (3/3)
Saving /work/src/yolov9/runs/vis/Crop_0030_030m_30c_3_4_0034411/stage1_Conv_features.png... (32/64)
Saving /work/src/yolov9/runs/vis/Crop_0030_030m_30c_3_4_0034411/stage2_Conv_features.png... (32/128)
Saving /work/src/yolov9/runs/vis/Crop_0030_030m_30c_3_4_0034411/stage3_RepNCSPELAN4_features.png... (32/256)
Saving /work/src/yolov9/runs/vis/Crop_0030_030m_30c_3_4_0034411/stage4_ADown_features.png... (32/256)
Saving /work/src/yolov9/runs/vis/Crop_0030_030m_30c_3_4_0034411/stage5_RepNCSPELAN4_features.png... (32/512)
Saving /work/src/yolov9/runs/vis/Crop_0030_030m_30c_3_4_0034411/stage6_ADown_features.png... (32/512)
Saving /work/src/yolov9/runs/vis/Crop_0030_030m_30c_3_4_0034411/stage7_RepNCSPELAN4_features.png... (32/512)
Saving /work/src/yolov9/runs/vis/Crop_0030_030m_30c_3_4_0034411/stage8_ADown_features.png... (32/512)
Saving /work/src/yolov9/runs/vis/Crop_0030_030m_30c_3_4_0034411/s

In [9]:
module_type = 'Conv'

if module_type not in ['Detect', 'CBLinear'] :
    print(module_type)
else:
    print('Correct')

Conv
